In [1]:
import importlib
import minas as mg
import pandas as pd

## Read Database

In [2]:
coords = ['RA_1', 'DEC_1']
filters = ['g_pstotal_0','i_pstotal_0','j0378_pstotal_0','j0395_pstotal_0','j0410_pstotal_0','j0430_pstotal_0','j0515_pstotal_0',
           'j0660_pstotal_0','j0861_pstotal_0','r_pstotal_0','u_pstotal_0','z_pstotal_0'
            ]

errors = ['e_g_PStotal','e_i_PStotal','e_J0378_PStotal','e_J0395_PStotal','e_J0410_PStotal','e_J0430_PStotal','e_J0515_PStotal',
          'e_J0660_PStotal','e_J0861_PStotal','e_r_PStotal','e_u_PStotal','e_z_PStotal'
            ]

# parameters of interest
stellar_params = ['teff', 'teff_err',
                  'logg', 'logg_err',
                  'feh', 'feh_err']


others = ['TILE_ID', 'snru', 'snrg', 'snrr', 'snri', 'snrz',
          'Dist', 'Plx', 'e_Plx', 'RPlx', 'RUWE']


In [3]:
rename_dict = {}

rename_dict = rename_dict | {key: value for key, value in zip(coords, ['RA', 'DEC'])}
rename_dict = rename_dict | {key: value for key, value in zip(filters, mg.FILTERS['SPLUS'])}
rename_dict = rename_dict | {key: value for key, value in zip(errors, mg.ERRORS['SPLUS'])}

In [4]:
columns =  coords + filters + errors + stellar_params + others

# Import the full dataframe with stars that have both SPLUS, GAIA DR3 and LAMOST(DR10 2.0/Low) data
stars_raw = mg.read_csv(f"splus_gL11.csv", usecols = columns)
stars_raw = stars_raw.rename(columns = rename_dict)
stars_raw.shape

(112023, 43)

In [5]:
for snr in ['snrg','snri', 'snrz']:
    stars_raw = stars_raw[stars_raw[snr] >= 10]
    
stars_raw = stars_raw[stars_raw['snrr'] >= 20]

stars_raw.shape

(93127, 43)

In [6]:
stars_raw = stars_raw[stars_raw['RUWE'] <= 1.4]
stars_raw.shape

(83440, 43)

In [7]:
stars_raw = stars_raw[stars_raw['RPlx'] >= 5]
stars_raw.shape

(74975, 43)

In [8]:
stars_raw.to_csv(f'input_splusgL11.csv',
                 index = False)

### FILTERS MAGNITUDE

In [9]:
import os
import pandas as pd
import minas as mg

# Caminho do arquivo de entrada
input_path = 'input_splusgL11.csv'
df_base = pd.read_csv(input_path)
df_base.set_index('TILE_ID', drop=True, inplace=True)

# Nome da coluna de índice (ou None para não usar)
index_col = 'TILE_ID'

# Survey e filtros
survey_filter = 'SPLUS'
filters = mg.FILTERS[survey_filter]


# Lista de parâmetros a serem processados
param_list = ['teff', 'logg', 'feh']

In [10]:
# Função utilitária para buscar o nome correto da coluna no DataFrame
def get_column(df, aliases):
    for col in aliases:
        if col in df.columns:
            return col
    raise KeyError(f'Nenhuma coluna encontrada para: {aliases}')

output_lines = []  # Lista para registrar os resumos dos filtros

for mag_err_type in ['_01', '_02']:
    print(f'\n--- Filtros para erro de magnitude: {mag_err_type} ---')
    max_err = 0.1 if mag_err_type == '_01' else 0.2
    df_common = df_base.copy()

    # Filtro de distância: remove valores inválidos de 'Dist'
    if 'Dist' in df_common.columns:
        df_common = df_common.dropna(subset=['Dist'])
        df_common = df_common[df_common['Dist'] != -9999]
        print(f'Objetos após filtro de Distância: {len(df_common)} objs')
        output_lines.append(f'Objetos após filtro de Distância: {len(df_common)} objs')

    error_columns = [f + '_err' for f in filters]

    # Filtros de erro de magnitude (aplicados uma vez)
    for col in error_columns:
        if col in df_common.columns:
            df_common = df_common[df_common[col] <= max_err]

    print(f'Objetos após filtro de erro de magnitude): {len(df_common)} objs')
    output_lines.append(f'--- Filtros para erro de magnitude: {mag_err_type} ---')
    output_lines.append(f'Objetos após filtro de erro de magnitude): {len(df_common)} objs')

    # Agora aplicar filtros específicos para cada parâmetro
    for param in param_list:
        aliases = mg.PARAM_ALIASES[param]
        df = df_common.copy()

        # Filtros específicos por parâmetro
        if param.lower() == 'teff':
            param_col = get_column(df, aliases)
            df = df[df[param_col] <= 8300]
            err_col_candidates = [c for c in df.columns if (c.lower().startswith('e_teff') or c.lower().endswith('teff_err'))]
            if err_col_candidates:
                err_col = err_col_candidates[0]
                df = df[df[err_col] < 300]
            df = df[df[param_col] != -9999]
            df = df.dropna(subset=[param_col])

        elif param.lower() in ['logg']:
            param_col = get_column(df, aliases)
            err_col_candidates = [c for c in df.columns if (c.lower().startswith('e_'+param.lower()) 
                                                            or c.lower().endswith(param.lower()+'_err'))]
            if err_col_candidates:
                err_col = err_col_candidates[0]
                df = df[df[err_col] < 0.4]
            df = df[df[param_col] != -9999]
            df = df.dropna(subset=[param_col])
        
        elif param.lower() in ['feh']:
            param_col = get_column(df, aliases)
            err_col_candidates = [c for c in df.columns if (c.lower().startswith('e_'+param.lower()) 
                                                            or c.lower().endswith(param.lower()+'_err'))]
            if err_col_candidates:
                err_col = err_col_candidates[0]
                df = df[df[err_col] < 0.4]
            df = df[df[param_col] != -9999]
            df = df.dropna(subset=[param_col])

        print(f'Objetos após filtro do erro do parâmetro {param}): {len(df)} objs')
        output_lines.append(f'Objetos após filtro do erro do parâmetro {param}): {len(df)} objs')

print()

# Salva as informações em um arquivo txt
with open('splusg_L11.txt', 'w') as f:
    for line in output_lines:
        f.write(line + '\n')
print('Resumo dos filtros salvo em splusg_L11.txt')


--- Filtros para erro de magnitude: _01 ---
Objetos após filtro de Distância: 69893 objs
Objetos após filtro de erro de magnitude): 61852 objs
Objetos após filtro do erro do parâmetro teff): 60849 objs
Objetos após filtro do erro do parâmetro logg): 60891 objs
Objetos após filtro do erro do parâmetro feh): 61211 objs

--- Filtros para erro de magnitude: _02 ---
Objetos após filtro de Distância: 69893 objs
Objetos após filtro de erro de magnitude): 66034 objs
Objetos após filtro do erro do parâmetro teff): 64630 objs
Objetos após filtro do erro do parâmetro logg): 64586 objs
Objetos após filtro do erro do parâmetro feh): 65139 objs

Resumo dos filtros salvo em splusg_L11.txt
